In [ ]:
import pandas as pd
from resolve_path import ajuste_path

In [ ]:
pathUtil = ajuste_path('data/util/')
pathInput = ajuste_path('data/input/')

df_acid = pd.read_csv(
    pathUtil + 'acidentes/acidentes_preparado.csv', encoding='utf-8', sep='#')

df_op = pd.read_csv(
    pathUtil + 'os/operacoes_preparado.csv', encoding='utf-8')

In [ ]:
colunas_os = list(df_op.columns)
colunas_os

In [ ]:
df_op['data_inicio'] = pd.to_datetime(df_op['data_inicio'])
df_op['data_fim'] = pd.to_datetime(df_op['data_fim'])
df_op['data_fim'] = df_op['data_fim'] + pd.Timedelta(days=1)
df_op.dtypes

In [ ]:
colunas_acid = df_acid.columns
colunas_acid

In [ ]:
df_acid['data'] = pd.to_datetime(df_acid['data'])
df_acid['id'] = df_acid['id'].astype(int)

In [ ]:
df_op['merge'] = False

colunas_os.append('merge')

colunas_acidente_na_op = set()
colunas_correto = set()

for idx, row in df_acid.iterrows():

    df_op.loc[
        ((df_op['data_inicio'] <= row['data'])
         & (df_op['data_fim'] > row['data']))  # mesma data
        & (df_op['no_pessoal'] == row['id']),  # mesma matricula
        'merge'
    ] = True

    if df_op[df_op['merge']].shape[0] == 0:
        continue

    if 'id_acidente' not in colunas_acidente_na_op:
        for key in row.index:
            colunas_acidente_na_op.add(key + '_acidente')
            df_op.loc[df_op['merge'], key + '_acidente'] = row[key]

        df_op[list(colunas_acidente_na_op)] = df_op[list(
            colunas_acidente_na_op)].astype(object)
        continue

    if df_op.loc[df_op['merge'], 'id_acidente'].isna().all():
        for key in row.index:
            colunas_acidente_na_op.add(key + '_acidente')
            df_op.loc[df_op['merge'], key + '_acidente'] = row[key]
    else:
        df_duplicatas = df_op.loc[df_op['merge'], colunas_os].reset_index()
        lim_sup = len(df_op)
        df_duplicatas.index = df_duplicatas.index + lim_sup + 1
        df_op = pd.concat([df_op, df_duplicatas], axis=0)
        df_op.loc[:lim_sup, 'merge'] = False

        for key in row.index:
            colunas_acidente_na_op.add(key + '_acidente')
            df_op.loc[df_op['merge'], key + '_acidente'] = row[key]

    df_op['merge'] = False

col_float = ['potencial_acidente',
             'latitude_acidente',
             'longitude_acidente',
             'id_acidente',
             ]

col_datetime = ['data_acidente']

df_op[col_float] = df_op[col_float].astype(float)

for col in col_datetime:
    df_op[col] = pd.to_datetime(df_op[col])

df_op.drop(columns=['merge', 'index'], axis=1, inplace=True)

In [ ]:
# df_op['descricao_do_acidente'] = df_op['descricao_do_acidente'].apply(
#     lambda x: x.replace(',', '.') if isinstance(x, str) else x)

In [ ]:
df_associacao = pd.read_csv(
    pathInput + "associa_acidentes_operacao_coordenadas_confianca.csv")

In [ ]:
df_associacao.info()

In [ ]:
# In column 'confianca', change values from 0,5 to float 0.5, if it is a string
df_associacao['Confianca'] = df_associacao['Confianca'].apply(
    lambda x: float(x.replace(',', '.')) if isinstance(x, str) else x)

df_associacao['Confianca'].value_counts()

In [ ]:
df_associacao = df_associacao[df_associacao['Confianca'] > 0]
# adicionar escolha pelo maior contador dentre as ops
for idx, row in df_associacao.iterrows():
    ordem = row['Ordem']
    op = row['Operação']
    id = row['ID']
    data = row['Dt']

    mask = ((df_op['ordem'] == ordem)
            & (df_op['operacao'] == op)
            & (df_op['no_pessoal'] == id)
            & ((df_op['data_inicio'] <= data)
               & (df_op['data_fim'] > data))
            )

    ops_ativas = df_op.loc[mask]
    if ops_ativas.shape[0] > 1:
        ultimo_lancamento = ops_ativas['contador'].max()
        mask = (df_op['contador'] == ultimo_lancamento) & mask
    for key in row.index:
        colunas_correto.add(key.lower() + '_correto')
        df_op.loc[mask, key.lower() + '_correto'] = row[key]

### Fazer testes para certificar onde e como '_correto' diferencia do '_acidente'

In [ ]:
df_teste = df_op[df_op['id_correto'].notna() | df_op['id_acidente'].notna()]

df_acidentes_possivel_errados = df_teste[df_teste['id_correto'].isna(
) & df_teste['id_acidente'].notna()]

### Corrigindo a associação a partir do link feit à mão

In [ ]:
df_op.loc[df_op['id_correto'].isna(),
          list(colunas_acidente_na_op)] = pd.NA

In [ ]:
df_op['desc_acid'] = df_op['descricao_acidente'].str.strip().str[:20]
df_op['desc_corr'] = df_op['descrição_correto'].str.strip().str[:20]
df_op['diferente'] = False
df_op.loc[df_op['desc_acid'] != df_op['desc_corr'], 'diferente'] = True
df_op.loc[df_op['id_acidente'].isna(), 'diferente'] = False
df_op.fillna({'diferente': False}, inplace=True)

df_op = df_op[~df_op['diferente']]

cols_intermediarias = ['desc_acid', 'desc_corr', 'diferente']
df_op.drop(columns=cols_intermediarias, inplace=True)

In [ ]:
df_teste = df_op[df_op['id_correto'].notna() | df_op['id_acidente'].notna()]

In [ ]:
df_op.drop(columns=colunas_correto, inplace=True)

In [ ]:
df_op.info()

In [ ]:
df_op.to_csv(pathUtil + 'dataset_associado.csv', encoding='utf-8', index=False)